GUIDED PROJECT FROM DATAQUEST 
Part of the course Data Analysis with Pandas: Intermediate

Last updated: November 16, 2017

In [1]:
import pandas

# Loading thanksgiving survey data into a dataframe called data
data = pandas.read_csv("thanksgiving.csv", encoding = "Latin-1")

In [2]:
#Checking the column index for the dataframe data

data.columns

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

In [3]:
import re

questions = data.columns.tolist()

# Function to retrieve the column index of the data set. 
# From the results above, the column indeces are also the survey questions.
# This function is designed to retrieve the column indeces to avoid possible typo errors when coding.
def get_col_index(regex, column_names = questions):
    for i, column in enumerate(column_names):
        if re.search(regex, column) is not None:
            col_index_list = i
            #print(i)          # For checking purposes only. If this is called more than once, the regex must be modified.
    col_index = column_names[col_index_list]
    return col_index

In [4]:
# First analysis for the guided project.
# Checking the response to "Do you celebrate Thanksgiving?"
celeb_thx = data[get_col_index("celebrate Thanksgiving")]

print()
print("Counting the responses to: Do you celebrate Thanksgiving?")
print(celeb_thx.value_counts())

# Filtering out "No" responses to "Do you celebrate Thanksgiving?"
thx_bool = celeb_thx == "Yes"
new_data = data[thx_bool]

# Note: new_data is a subset of data with the No response rows filtered out.


Counting the responses to: Do you celebrate Thanksgiving?
Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64


In [5]:
# Generic function that searches for the occurrence of regex within the column indeces in the list questions
# and adds the indeces to a list.
# The code was originally meant for the dessert analysis.
# But it was found that this piece of code may be reused for other analysis.
def find_and_list(regex, column_names = questions):
    number_index = []
    for i, column in enumerate(column_names):
        if re.search(regex, column):
            number_index.append(i)
    return number_index

# Another generic function.
# This is for printing the unique items of a specific dataframe column (namely, new_data) 

def explore_contents(list_index, column_names = questions, main_data = new_data):
    for each_index in list_index:
        print(each_index, ": ", main_data[column_names[each_index]].unique())

In [6]:
# Second analysis for the guided project.
# Main dish question - Checking the count for each answer
main_dish = new_data["What is typically the main dish at your Thanksgiving dinner?"]

print("Counting the responses to: What is typically the main dish at your Thanksgiving dinner?")
print(main_dish.value_counts())
print()

# Selecting only the rows where Tofurkey is the main dish
tofurkey_bool = main_dish == "Tofurkey"
all_tofurkey = new_data[tofurkey_bool]

# Selecting the column for the gravy question for tofurkey
tofurkey_gravy = all_tofurkey["Do you typically have gravy?"]
print("Counting the number of people who eat tofurkey with gravy")
print(tofurkey_gravy.value_counts())

Counting the responses to: What is typically the main dish at your Thanksgiving dinner?
Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

Counting the number of people who eat tofurkey with gravy
Yes    12
No      8
Name: Do you typically have gravy?, dtype: int64


In [7]:
# Third analysis for the guided project.
# In this analysis, the number of people eating apple pie, pumpkin pie or pecan pie was determined.

# Creating a new function for the pie questions
# Sample question: What type of pie is typically served at your Thanksgiving dinner? Please select all that apply. Apple
# For this analysis, only three types of pie were considered: apple, pumpkin and pecan.
# The function get_col_index was not used because a test showed that the keyword Apple yielded two results.
# The way the question was phrased, it was more convenient to use endswith() instead of re.search().

def pie_question(regex, question_set = questions):
    for i, column in enumerate(question_set):
        if column.endswith(regex):
            pie_index = i
    survey_question = question_set[pie_index]
    return survey_question

# Retrieving results for the pie question columns
apple = new_data[pie_question("Apple")]
pumpkin = new_data[pie_question("Pumpkin")]
pecan = new_data[pie_question("Pecan")]

# Creating Boolean vectors for each pie type question. Need to check which rows are null. 
apple_isnull = pandas.isnull(apple)
pumpkin_isnull = pandas.isnull(pumpkin)
pecan_isnull = pandas.isnull(pecan)

# The number of people who eat pies will be determined by the expression
# apple is null & pumpkin_isnull & pecan_isnull, evaluated per row.
# When the per row result is True, then the survey respondent does not eat pie.
# Conversely, when the row result is False, then the survey respondent eats pie.
# Therefore, the number of people eating pie is equal to the number of False values.
ate_pies = apple_isnull & pumpkin_isnull & pecan_isnull

print(ate_pies.value_counts())

False    876
True     104
dtype: int64


In [8]:
# Fourth analysis for the guided project
# The age column of the original data set is an age range. This column is a string value.
# This function converts this string value to the integer value of the first encountered age.
def convert_age(value_age):
    if pandas.isnull(value_age):     # line is needed to avoid errors when the survey response is null
        return None
    else:
        age_list = value_age.split(' ')
        age_new = age_list[0]
        if (age_new.endswith("+")):
            age_new = "60"     # there is only one item with '+' - the age category of 60+
        age_new = int(age_new)
        return age_new

# Displaying the unique values for the original column    
print("ORIGINAL DATA SET")
print(new_data["Age"].unique())
print()
    
# Converting the Age column to a numerical equivalent.
age_column = new_data["Age"]
age_numerical = age_column.apply(convert_age)

print("CONVERTED DATA SET")
print(age_numerical.unique())

ORIGINAL DATA SET
['18 - 29' '30 - 44' '60+' '45 - 59' nan]

CONVERTED DATA SET
[ 18.  30.  60.  45.  nan]


In [9]:
# Fifth analysis for the guided project.
# Similar to the Age column, the income column is also a range of values.
# The raw values is in string form, not in integer or float. The function converts the range into an integer,
# by taking the first value within the range and converting it to integer.
def convert_value_income(income):
    if pandas.isnull(income):     # line is needed to avoid error when the survey response is null
        return None
    else:
        income_list = income.split(" ")
        income = income_list[0]
        if income == "Prefer":     # line is needed to avoid error when the survey response is "Prefer not to..."
            return None 
        else:
            income = re.sub("\$","", income)
            income = re.sub(",", "", income)
            income = int(income)
            return income

# Accessing the column with index title of
# How much total combined money did all members of your HOUSEHOLD earn last year?
annual_income_raw = new_data[get_col_index("earn last year?")]

# Converting the income range to an approximate integer value (based on the low of the range)
int_income = annual_income_raw.apply(convert_value_income)

# Performing the analysis
print()
print("Statistics for the annual income survey results:")
int_income.describe()


Statistics for the annual income survey results:


count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: How much total combined money did all members of your HOUSEHOLD earn last year?, dtype: float64

With the exception of the value for count, the resulting statistics are rough approximations only, because no exact value of the income was given by the participants. The survey response was either a range of values or no value at all. For the range of values, only the lowest value of the range was retained while the non-numerical responses were ignored.

In [10]:
# Sixth analysis for the guided project.
# Proving the hypothesis that people with higher income are more likely to have thanksgiving at their house.

# The column for How far will you travel for Thanksgiving?
travel_distance = new_data[get_col_index("travel for Thanksgiving?")]

# Displaying the unique responses and their corresponding count
print()
print("THESE ARE THE RESULTS FOR ALL RESPONDENTS:")
print(travel_distance.value_counts())

income_filter = int_income > 150000
travel_15k = travel_distance[income_filter]

print()
print("THESE ARE THE RESULTS FOR RESPONDENTS WITH INCOME HIGHER THAN $150K")
print(travel_15k.value_counts())



THESE ARE THE RESULTS FOR ALL RESPONDENTS:
Thanksgiving is happening at my home--I won't travel at all                         396
Thanksgiving is local--it will take place in the town I live in                     276
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    197
Thanksgiving is out of town and far away--I have to drive several hours or fly       82
Name: How far will you travel for Thanksgiving?, dtype: int64

THESE ARE THE RESULTS FOR RESPONDENTS WITH INCOME HIGHER THAN $150K
Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64


HYPOTHESIS: People with higher income are more likely to have Thanksgiving at their house.

For this exercise, the definition for higher income is income greater than $ 150,000. 

Given this filter, it was found that:


People having thanksgiving at home = 49

People having thanksgiving within their own town = 25

People driving a few hours or less = 16

People driving several hours or flying = 12


Based on the results above, it is found that the hypothesis is true. 

In [11]:
# Seventh analysis for the guided project.
# Proving the hypothesis that younger people favored spending thanksgiving with friends.

friends = get_col_index("friends on Thanksgiving night?")
friendsgiving = get_col_index("Friendsgiving")

# Taking a subset from new_data.
# Note that the original values for column index "Age" is a string of age ranges.
# The "Age" column must be replaced by the values of age_numerical.
# But operating this on the original data set of new_data triggers a SettingWithCopyWarning.
# To avoid this warning, a subset of the dataframe is created.
# The subset has the friends column, friendsgiving column, and the original "Age" column.
# Any changes to column "Age" will be operated on this subset.
subset_data = new_data[[friends, friendsgiving, "Age"]].copy()
subset_data["Age"] = age_numerical

# Changing the column names
# The column names are too long that it is not easy to view the pivot results with the original column names.
subset_data = subset_data.rename(columns = {friends: "Meet with friends"})
subset_data = subset_data.rename(columns = {friendsgiving: "Friendsgiving"})

# Doing the analysis using pivot table
# While both tables give the same results, both were done because the guided project required both steps.
pivot_results1 = subset_data.pivot_table(index = "Meet with friends", values = "Age", columns = "Friendsgiving")
pivot_results2 = subset_data.pivot_table(index = "Friendsgiving", values = "Age", columns = "Meet with friends")

print()
print("PIVOT RESULTS FOR MEETING WITH FRIENDS AS INDEX AND FRIENDSGIVING AS COLUMN")
display(pivot_results1)
print()
print("PIVOT RESULTS FOR FRIENDSGIVING AS INDEX AND MEETING WITH FRIENDS AS COLUMN")
display(pivot_results2)




PIVOT RESULTS FOR MEETING WITH FRIENDS AS INDEX AND FRIENDSGIVING AS COLUMN


Friendsgiving,No,Yes
Meet with friends,,
No,42.283702,37.010526
Yes,41.475410,33.976744



PIVOT RESULTS FOR FRIENDSGIVING AS INDEX AND MEETING WITH FRIENDS AS COLUMN


Meet with friends,No,Yes
Friendsgiving,,
No,42.283702,41.475410
Yes,37.010526,33.976744


It was hypothesized that younger respondents tend to spend or wish to spend thanksgiving with their friends.

Based on the average value results above, those who opted to spend thanksgiving with their friends are in their 30's (not too young nor too old). 

Again, the results should consider that the ages of participants were mere approximations only and that the age registered within the data set is the lowest value of the age range.

In [12]:
# Eight Analysis: Suggested next step for the guided project.
# Determine the most common dessert.

# Exploring the contents of all columns with the keyword "dessert" in the column index
# The list of index numbers extracted from the list questions for survey questions with the term "dessert"
dessert_index = find_and_list("dessert")


# Displaying the values of the unique responses for the dessert questions 
explore_contents(dessert_index)

39 :  [nan 'Apple cobbler']
40 :  [nan 'Blondies']
41 :  [nan 'Brownies']
42 :  [nan 'Carrot cake']
43 :  ['Cheesecake' nan]
44 :  ['Cookies' nan]
45 :  [nan 'Fudge']
46 :  ['Ice cream' nan]
47 :  [nan 'Peach cobbler']
48 :  [nan 'None']
49 :  [nan 'Other (please specify)']
50 :  [nan 'Jelly roll, sweet cheeseball, chocolate dipped berries' 'Pie'
 'Sparkling Apple Cider' 'Pumpkin Pie' 'pies' 'Lefse' 'eclair' 'as above'
 'Pie is dessert' 'chocolate mousse' 'pine nut cake ' 'Pumpkin pie.'
 'Pie, pumpkin' 'we stick to pie' 'choc. cake' 'pumpkin pie' 'pie'
 'Pies and dream whip' 'Pumpkin pie' 'Lefse or Krumkakke' 'pumpkin bars'
 'Banana Pudding' 'pumpkin roll' 'chocolate cake'
 'THE PIE YOU JUST ASKED ABOUT. '
 'pies: pumpkin with cie cream vanilla, apple, pecan'
 'With pie listed above' 'cheesecake.' 'Restaurant with various choices.'
 'pumpkin pie. why ask this twice?'
 'Chocolate Cake with Marshmellows, Caremal Cake'
 'Triple Treat (butterscotch, cream cheese mix, and nut crust' 'Just p

In [13]:
# Here, we make an assumption. Responses for des_index 50 are likely to be less than the responses for
# Apple cobbler, Blondies, Brownies, Carrot cake, Cheesecake,Cookies, Fudge, Ice Cream and Peach Cobbler
# Therefore, des_index 50 will be ignored. des_index 48 is also ignored because no dessert was chosen here.
# des_index 49 is also ignored because the "Other" was not specified.

subset_dessert_index = dessert_index[0:9]

for des_index in subset_dessert_index:
    response_count = new_data[questions[des_index]].value_counts().to_dict()
    print(response_count)

{'Apple cobbler': 110}
{'Blondies': 16}
{'Brownies': 128}
{'Carrot cake': 72}
{'Cheesecake': 191}
{'Cookies': 204}
{'Fudge': 43}
{'Ice cream': 266}
{'Peach cobbler': 103}


Based on the results above, the most common dessert for Thanksgiving is ice cream (with 266 responses).

In [14]:
# Ninth Analysis: Suggested next step for the guided project
# Identify how many people work on thanksgiving.

# Getting the column index for the question Will your employer make you work on Black Friday?
black_friday = get_col_index("work on Black Friday")

# Counting the number of yes responses
print()
print("NUMBER OF RESPONDENTS WORKING ON BLACK FRIDAY - YES COUNT ONLY")
print(new_data[black_friday].value_counts())


NUMBER OF RESPONDENTS WORKING ON BLACK FRIDAY - YES COUNT ONLY
Yes              43
No               20
Doesn't apply     7
Name: Will you employer make you work on Black Friday?, dtype: int64


Based on the results above, there are 43 respondents who will be working on Black Friday (as opposed to the 27 who will not work).

In [15]:
# Tenth Analysis: Suggested next step for the guided project
# Figure out the most common complete meal people eat

# A complete meal consists of main dish, side dish and dessert. 
# For this data set, condiments were separately included in the survey. Condiments include sauce, gravy, dressing, stuffing.
# Additionally, pie types were considered separate from desserts.
# For simplicity of analysis, the condiment columns will be ignored.
# The pies will also be included in the dessert category.

# Analyzing contents of all survey questions with the term "main dish"

main_dish = find_and_list("main dish")

explore_contents(main_dish)

2 :  ['Turkey' 'Tofurkey' 'Other (please specify)' 'Ham/Pork' 'Turducken'
 'Roast beef' nan 'Chicken' "I don't know"]
3 :  [nan 'Turkey and Ham' 'Varies' 'some kind of lentil or vegetable stew'
 'Prime Rib' 'fish' 'A wild game bird which changes every year.'
 'always different' 'Restaurant with various choices.' 'gumbo'
 'alternate turkey & ham' 'seafood' 'Chicken Dressing'
 'Both turkey and a vegetarian nut loaf'
 'This year we are having baked lasagna, diff every year' 'Steak'
 'It varies, ham or turkey or boston butt' 'No predictable main dish'
 'Venison' 'turkey, ham. vegetarian for me.' 'Homemade vegan entree'
 'head cheese' 'Lentil Loaf' 'changes every year'
 'Turkey and Vegetarian Turkey'
 'Turkey & Ham or Seafood Hotdish (a secret family recipe).' 'steak'
 'A turkey and a ham. Always.' 'prime rib' 'goose'
 'Varies significantly by year as one group of people we tend to be with are vegetarian'
 'salmon' 'Duck']
4 :  ['Baked' 'Roasted' 'Fried' 'Other (please specify)' "I don't kn

In [16]:
# Meat Type: Index 2, 3
# Cook Method: Index 4, 5
# Index 3 and 5 will be ignored for this analysis because of the diversity of the data.
# It is assumed that the diversity will make it less likely for it to become part of a "most common" complete meal

# Preparation - main dish
main_dish_subset = [2, 4]

In [17]:
# Analyzing contents of all survey questions with the term "side dishes"

side_dish = find_and_list("side dishes")

explore_contents(side_dish)

11 :  [nan 'Brussel sprouts']
12 :  ['Carrots' nan]
13 :  [nan 'Cauliflower']
14 :  [nan 'Corn']
15 :  [nan 'Cornbread']
16 :  [nan 'Fruit salad']
17 :  ['Green beans/green bean casserole' nan]
18 :  ['Macaroni and cheese' nan]
19 :  ['Mashed potatoes' nan]
20 :  [nan 'Rolls/biscuits']
21 :  [nan 'Squash']
22 :  [nan 'Vegetable salad']
23 :  ['Yams/sweet potato casserole' nan]
24 :  [nan 'Other (please specify)']
25 :  [nan 'Asian vinagrette salad' 'broccoli/cheese sauce' 'depends'
 'squash soufflet' 'roasted vegatables' 'Okra!' 'fresh green peas'
 'Broccoli' 'Peas, sometimes cranberry jello mold' 'Rice pudding'
 'Cajun Cressing (Crayfish, gorund beef & rice, etc.)' 'collard greens'
 'potato salad' 'pumpkin bisque'
 'Peas with mushrooms. Sparkly white wine. sweet gherkin pickles, pumpkin pie'
 'plain sweet potatoes, brocolli, relish tray, oyster dressing'
 'baked sweet potatoes, broccoli & cheese sauce' 'Asparagus'
 'Turnip greens' 'Antipasta' 'Rice'
 "I love brussel sprouts, but my mo

In [18]:
# Excluding index 24 and 25.
# Index 24 essentially has no data.
# Index 25 is too diverse.

# Preparation - side dish
side_dish_subset = side_dish[0:len(side_dish)-2]

In [19]:
# Analyzing contents of all survey questions with the term "pie"

pie = find_and_list("pie")
explore_contents(pie)

26 :  ['Apple' nan]
27 :  [nan 'Buttermilk']
28 :  [nan 'Cherry']
29 :  [nan 'Chocolate']
30 :  [nan 'Coconut cream']
31 :  [nan 'Key lime']
32 :  [nan 'Peach']
33 :  [nan 'Pecan']
34 :  [nan 'Pumpkin']
35 :  [nan 'Sweet Potato']
36 :  [nan 'None']
37 :  [nan 'Other (please specify)']
38 :  [nan 'Derby, Japanese fruit' 'Blueberry pie' 'Strawberry Rhubarb'
 'Lemon mirangue' 'Chess' 'We serve pine nut cake not pie'
 'lemon maraigne, minch meat' 'mincemeat' 'Cheesecak' 'Pecan, Mincemeat'
 'Peanut butter' 'grass hopper' 'rhubarb' 'minced meat' 'Shoo Fly Pie '
 'Squash' 'raspberry' 'Berry' 'Custard' 'Mincemeat ' 'Lemon'
 'peanut butter' 'pumpkin cheesecake' 'Mint Torte' 'blueberry' 'Pecan '
 'pie with ice cream and whipped cream' 'lemon merang' 'lemon meringue'
 'pecan' 'Cherry' 'Mince meat' 'rhubarb strawberry' 'apricot'
 'banana cream' 'German chocolate pecan' 'Cheesecake ' 'sweet potato pie'
 'Strawberry cream' 'Concord grape' 'Chess ' 'Lemon Meringue ' 'Shoo fly'
 'Blueberry' 'cherry ch

In [20]:
# Excluding index 36 through 38.
# Indeces 36 & 37 essentially has no data.
# Index 38 results are too diverse.

# Preparation - Pie
pie_subset = pie[0: len(pie)-3]

In [21]:
# Merging all indeces
new_index = main_dish_subset + side_dish_subset + pie_subset + subset_dessert_index

# Creating a subset of the column indeces taken from new_index
question_subset = []
for ni in new_index:
    text = questions[ni]
    question_subset.append(text)

# Creating a subset of new_data with the column names from question_subset
complete_meal_data = new_data[question_subset].copy()

# Converting the row of the dataframe complete_meal_data to a single string expression
# and creating a new series from these string expressions
complete_meal_list = []

for xx in range (0, len(complete_meal_data)):
    row_list = complete_meal_data.iloc[xx].tolist()     # Converting a row to a list
    row_string = ",".join(map(str, row_list))           # Converting the list to a single string expression
    complete_meal_list.append(row_string)               # Saving the single string expression to a list

complete_meal_series = pandas.Series(data = complete_meal_list)     # Converting the single string expression list to a series

complete_meal_count = complete_meal_series.value_counts()

# Displaying the top 5 results
print("TOP FIVE COMPLETE MEAL RESULTS")
complete_meal_count.head()



TOP FIVE COMPLETE MEAL RESULTS


nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan                                                                                             6
Turkey,Roasted,nan,nan,nan,nan,nan,nan,Green beans/green bean casserole,nan,Mashed potatoes,Rolls/biscuits,nan,nan,Yams/sweet potato casserole,nan,nan,nan,nan,nan,nan,nan,nan,Pumpkin,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan      4
Turkey,Roasted,nan,nan,nan,nan,nan,nan,Green beans/green bean casserole,nan,Mashed potatoes,Rolls/biscuits,nan,nan,Yams/sweet potato casserole,nan,nan,nan,nan,nan,nan,nan,Pecan,Pumpkin,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan    4
Turkey,Roasted,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan                                                                                      4
Turkey,Baked,nan,nan,nan,Corn,nan,nan,Green beans/green bean casserole,nan,Mashe

While the results above is cumbersome to read, it can be deduced by careful examination that the top result belongs to a string of all nan values. This string appears 6 times. Recalling that columns with very diverse responses were ignored, the nan value string simply means that the respondent chose a different dish from the columns that were considered. Without going further into the analysis, it will be assumed that a deeper inspection will still show a very diverse combination of dishes for the main dish, side dish and dessert. Hence, the responses for these entries will not reflect the most common complete meal for thanksgiving. 

The next top result belongs to three string values which will be summarized as follows:

DISH COMBINATION 1: Roasted Turkey

DISH COMBINATION 2: Roasted Turkey, Green beans/green bean casserole, Mashed potatoes, Rolls/biscuits, Yams/sweet potato casserole, Pecan Pie, Pumpkin Pie

DISH COMBINATION 3: Roasted Turkey, Green beans/green bean casserole, Mashed potatoes, Rolls/biscuits, Yams/sweet potato casserole, Pumpkin Pie


All four string combinations have a frequency of 4. 

Dish Combination 1 will be ignored because "Roasted Turkey" is only a main dish. Once more, the respondents for this survey may have added side dishes and desserts from the columns that were ignored. 

Thus, the most common complete meal belongs to DISH COMBINATION 2 AND 3, which are almost identical to each other. The only difference between the two is that DISH COMBINATION 2 has Pecan Pie in addition to Pumpkin Pie.


In [22]:
# Eleventh Analysis - suggested next step for the guided project
# Find regional patterns in the dinner menu

# Exploring the responses to "US Region".

region = new_data["US Region"]
region_list = region.unique().tolist()

def mdmeat_analysis(filtered_results):
    print("MAIN DISH MEAT")
    frequency_count = filtered_results[questions[2]].value_counts()
    print(frequency_count.to_dict())
    print()
    return()

def mdcook_analysis(filtered_results):
    print("MAIN DISH - COOKING METHOD")
    frequency_count = filtered_results[questions[4]].value_counts()
    print(frequency_count.to_dict())
    print()
    return()

def multiple_col_counter(f_results, idx_list, column_names = questions):
    for idx in idx_list:
        frequency_count = f_results[column_names[idx]].value_counts().to_dict()
        print(frequency_count)
    return()

def sidedish_analysis(filtered_results, sd_index = side_dish_subset):
    print("SIDE DISHES")
    multiple_col_counter(filtered_results, sd_index)
    print()
    return()

def pies_analysis(filtered_results, p_list = pie_subset):
    print("PIES")
    multiple_col_counter(filtered_results, p_list)
    print()
    return()

def desserts_analysis(filtered_results, d_list = subset_dessert_index):
    print("DESSERTS")
    multiple_col_counter(filtered_results, d_list)
    print()
    return()

def per_column_analysis(end_num, column_in_list, column_in_df, nd = new_data):
    for i in range (0, end_num):
        boolean_filter = column_in_df == column_in_list[i]
        filtered_column = new_data[boolean_filter]
        print(column_in_list[i])
        mdmeat_analysis(filtered_column)
        mdcook_analysis(filtered_column)
        sidedish_analysis(filtered_column)
        pies_analysis(filtered_column)
        desserts_analysis(filtered_column)
    return()

per_column_analysis(len(region_list)-1, region_list, region)

Middle Atlantic
MAIN DISH MEAT
{'Turkey': 130, 'Tofurkey': 5, 'Other (please specify)': 4, 'Roast beef': 2, 'Ham/Pork': 2, 'Chicken': 1, 'Turducken': 1}

MAIN DISH - COOKING METHOD
{'Roasted': 71, 'Baked': 62, 'Other (please specify)': 7, 'Fried': 5}

SIDE DISHES
{'Brussel sprouts': 41}
{'Carrots': 45}
{'Cauliflower': 25}
{'Corn': 77}
{'Cornbread': 33}
{'Fruit salad': 26}
{'Green beans/green bean casserole': 92}
{'Macaroni and cheese': 20}
{'Mashed potatoes': 130}
{'Rolls/biscuits': 106}
{'Squash': 44}
{'Vegetable salad': 33}
{'Yams/sweet potato casserole': 99}

PIES
{'Apple': 106}
{'Buttermilk': 2}
{'Cherry': 17}
{'Chocolate': 16}
{'Coconut cream': 9}
{'Key lime': 1}
{'Peach': 4}
{'Pecan': 31}
{'Pumpkin': 115}
{'Sweet Potato': 21}

DESSERTS
{'Apple cobbler': 23}
{}
{'Brownies': 18}
{'Carrot cake': 17}
{'Cheesecake': 34}
{'Cookies': 40}
{'Fudge': 8}
{'Ice cream': 51}
{'Peach cobbler': 10}

East South Central
MAIN DISH MEAT
{'Turkey': 50, 'Other (please specify)': 4, 'Roast beef': 1, 'H

()

For the main dish (meat type), turkey consistently tops the count for any of the regions considered. This result confirms the stereotype of a thanksgiving dinner having turkey as a staple.

For the method of cooking, the top two answers are "roasted" and "baked" with more regions biased towards "baked" more than "roasted". Somehow, the Hollywood stereotype of a thanksgiving turkey being roased is proven wrong by the survey results.

For the side dishes, all regions seem to lean towards green beans/ green bean casserole, mashed potatoes, rolls / biscuits or yam / sweet potato casserole. 

For the pies, the regional results lean towards apple pie and pumpkin pie, although some regions favored pecan pie more than applie pie or pumpkin pie.





In [23]:
# Twelfth Analysis - Suggested next steps for the guided project
# Find age, gender, and income based patterns in dinner menu

# Analyzing by age
age_column = new_data["Age"]
age_list = age_column.unique().tolist()
age_length = len(age_list)-1

per_column_analysis(age_length, age_list, age_column)

18 - 29
MAIN DISH MEAT
{'Turkey': 162, 'Tofurkey': 6, 'Ham/Pork': 6, 'Chicken': 3, 'Roast beef': 3, "I don't know": 3, 'Other (please specify)': 2}

MAIN DISH - COOKING METHOD
{'Baked': 108, 'Roasted': 51, 'Fried': 13, "I don't know": 9, 'Other (please specify)': 4}

SIDE DISHES
{'Brussel sprouts': 30}
{'Carrots': 61}
{'Cauliflower': 27}
{'Corn': 111}
{'Cornbread': 71}
{'Fruit salad': 39}
{'Green beans/green bean casserole': 126}
{'Macaroni and cheese': 61}
{'Mashed potatoes': 160}
{'Rolls/biscuits': 151}
{'Squash': 26}
{'Vegetable salad': 44}
{'Yams/sweet potato casserole': 105}

PIES
{'Apple': 103}
{'Buttermilk': 9}
{'Cherry': 32}
{'Chocolate': 35}
{'Coconut cream': 6}
{'Key lime': 15}
{'Peach': 11}
{'Pecan': 69}
{'Pumpkin': 146}
{'Sweet Potato': 31}

DESSERTS
{'Apple cobbler': 32}
{'Blondies': 7}
{'Brownies': 48}
{'Carrot cake': 16}
{'Cheesecake': 39}
{'Cookies': 66}
{'Fudge': 15}
{'Ice cream': 73}
{'Peach cobbler': 27}

30 - 44
MAIN DISH MEAT
{'Turkey': 197, 'Ham/Pork': 12, 'Other 

()

There is no marked difference between the main dish meat preference among different ages. Similar to the per region results, turkey is the top choice for main dish meat.

For the cooking method, only the respondents aged 60+ preferred roasting over baking. The rest preferred baking.

Similar to the main dish meat results, the top result is consistent over all ages - mashed potatoes is more preferred.

Pumpkin pie is still the main choice for all age groups, followed by apple pie and pecan pie.

For the desserts, the top choice is still ice cream for all ages. Cookies come second for age ranges from 18 to 59. Majority of the people aged 60+ preferred cheesecake over cookies.



In [24]:
# Analyzing by gender

gender_index = get_col_index("gender")
gender_column = new_data[gender_index]
gender_list = ["Male", "Female"]

per_column_analysis(2, gender_list, gender_column)

Male
MAIN DISH MEAT
{'Turkey': 383, 'Ham/Pork': 13, 'Other (please specify)': 13, 'Tofurkey': 7, 'Roast beef': 6, 'Chicken': 6, "I don't know": 2, 'Turducken': 2}

MAIN DISH - COOKING METHOD
{'Baked': 208, 'Roasted': 171, 'Fried': 25, 'Other (please specify)': 18, "I don't know": 10}

SIDE DISHES
{'Brussel sprouts': 65}
{'Carrots': 129}
{'Cauliflower': 44}
{'Corn': 240}
{'Cornbread': 119}
{'Fruit salad': 92}
{'Green beans/green bean casserole': 307}
{'Macaroni and cheese': 93}
{'Mashed potatoes': 366}
{'Rolls/biscuits': 335}
{'Squash': 78}
{'Vegetable salad': 84}
{'Yams/sweet potato casserole': 279}

PIES
{'Apple': 222}
{'Buttermilk': 12}
{'Cherry': 52}
{'Chocolate': 58}
{'Coconut cream': 21}
{'Key lime': 24}
{'Peach': 22}
{'Pecan': 153}
{'Pumpkin': 319}
{'Sweet Potato': 72}

DESSERTS
{'Apple cobbler': 65}
{'Blondies': 9}
{'Brownies': 62}
{'Carrot cake': 47}
{'Cheesecake': 81}
{'Cookies': 97}
{'Fudge': 23}
{'Ice cream': 128}
{'Peach cobbler': 56}

Female
MAIN DISH MEAT
{'Turkey': 452, 

()

Similar to the result for age ranges, gender does not affect the preference for turkey.

The preferred method of cooking for both gender is baked, i.e., the cooking method is not affected by gender either.

Once more, the preferred side dishes seem to be mashed potatoes, rolls / biscuits, green beans / green bean casserole and yam / sweet potato casserole (arranged from the most frequent to the less frequent). The results show that there is also no gender bias with respect to the choice of side dishes.

Consistent with the findings in other analysis, the top choice for both gender is pumpkin pie, apple pie and pecan pie. Again, gender does not affect the choice of pies.

The top choices for dessert are still ice cream, cookies and cheesecake. But females marginally prefer cheesecake over cookies (in contrast with males preferring cookies over cheesecake).

In [25]:
# Analyzing by annual income

#print(annual_income_raw)

annual_income_list = annual_income_raw.unique().tolist()
annual_income_list.remove("Prefer not to answer")
annual_income_length = len(annual_income_list)-1

per_column_analysis(annual_income_length, annual_income_list, annual_income_raw)

$75,000 to $99,999
MAIN DISH MEAT
{'Turkey': 112, 'Ham/Pork': 8, 'Other (please specify)': 5, 'Tofurkey': 2}

MAIN DISH - COOKING METHOD
{'Baked': 62, 'Roasted': 48, 'Other (please specify)': 9, 'Fried': 6, "I don't know": 2}

SIDE DISHES
{'Brussel sprouts': 25}
{'Carrots': 31}
{'Cauliflower': 15}
{'Corn': 57}
{'Cornbread': 29}
{'Fruit salad': 26}
{'Green beans/green bean casserole': 87}
{'Macaroni and cheese': 25}
{'Mashed potatoes': 113}
{'Rolls/biscuits': 103}
{'Squash': 21}
{'Vegetable salad': 22}
{'Yams/sweet potato casserole': 74}

PIES
{'Apple': 64}
{'Buttermilk': 6}
{'Cherry': 13}
{'Chocolate': 15}
{'Coconut cream': 3}
{'Key lime': 3}
{'Peach': 2}
{'Pecan': 45}
{'Pumpkin': 95}
{'Sweet Potato': 13}

DESSERTS
{'Apple cobbler': 15}
{'Blondies': 3}
{'Brownies': 13}
{'Carrot cake': 10}
{'Cheesecake': 24}
{'Cookies': 29}
{'Fudge': 7}
{'Ice cream': 32}
{'Peach cobbler': 8}

$50,000 to $74,999
MAIN DISH MEAT
{'Turkey': 118, 'Other (please specify)': 3, 'Roast beef': 2, 'Ham/Pork': 2, '

()

Turkey is consistently the top response and annual income has no effect on this preference.

For the method of cooking, the general preference is to either bake or roast the main dish.

The top 4 choices for the side dishes are still mashed potatoes, rolls / biscuits, green beans / green bean casserole and yam / sweet potato casserole, regardless of income.

Likewise, for the pie, the top 3 choices are pumpkin pie, apple pie and pecan pie.

